# Spline-Interpolation am Biegebalken

Der Biegebalken mit einseitiger fester Einspannung ist ein Standardlastfall, der Ihnen aus der technischen Mechanik bekannt ist.

<img src="../images/spline_interpolation.png" width="400" height="315" />

Messungen der Biegelinie an verschiedenen Punkten ergaben folgende Werte:

| $x [m]$ | 0   | 0.5 | 1 | 1.5 | 2 |
|:-------|:---|:---|:---|:---|:---|
| $z [m]$ | 0.00 | 0.0125 | 0.05 | 0.1125 | 0.2  |

Die Messwerte sollen nun interpoliert werden (**Hierbei keine fertige MATLAB Spline-Funktion benutzen - Finger weg von der Spline-Toolbox!**). Dazu wird jedes Teilintervall mit einem geeigneten Polynom genähert. Der Einfachheit halber werden hier
Polynome vom Grad zwei verwendet.

## Aufgabe 1

Konstruieren sie eine quadratische Splinefunktion durch die Anfangs- und Endpunkte der Teilintervalle (alles Bleistift und Papier!).
Bestimmen sie die Koeffizienten der gesuchten Parabeln $z_i(x) = a_i + b_i(x-x_i) + c_i(x-x_i)^2, i \in [0,2]$.

**Hinweis:** Zur eindeutigen Bestimmung der einzelnen Polynome sind drei Bedingungen notwendig. Zwei sind durch
den Anfangs- und Endpunkt des jeweiligen Intervalls gegeben. Um Knicke im gesamten Spline zu vermeiden
wird zusätzlich gefordert, dass die ersten Ableitungen an den Intervallgrenzen übereinstimmen
(dritte Bedingung). Welche Randbedingung ist für die erste Ableitung an der Einspannung zu wählen?


## Aufgabe 2

 - Implementieren Sie eine Matlab-Funktion `function zfine = quadraticSpline(x, z, xfine, bc)`, die für gegebene x- und z-Werte sowie Randbedingung auf einem fein aufgelösten Interval, definiert durch xfine, die entsprechenden z-Werte entlang des Splines berechnet.

In [14]:
%%file quadraticSpline.m
% return interpolated z from quadratic interpolation of input x-z-value pairs
function zfine = quadraticSpline(x, z, xfine, bc)

    if(max(xfine)> max(x) || min(xfine)<min(x))
        msgID = 'quadraticSpline:xfineOutofbounds';
        msg = 'A value in xfine is outside the range of x.';
        exception = MException(msgID,msg);
        throw(exception);
    end
    
    if(max(size(x) ~= size(z)))
        msgID = 'quadraticSpline:xzdiffer';
        msg = 'x and z dimensions do not agree.';
        exception = MException(msgID,msg);
        throw(exception);
    end

    b = [z, zeros(size(z))]';
    b(length(z)+1) = bc;
    A = zeros(2*length(z),3*length(z));
    for i = 1:length(z)
        A(i,(i-1)*3+1) = 1;
        A(i,(i-1)*3+2) = x(i);
        A(i,(i-1)*3+3) = x(i).^2;
    end
    A(length(z)+1,2) = 1;
    A(length(z)+1,3) = 2*x(1);
    for i = 2:length(z)
        A(i + length(z),(i-1)*3+2) = 1;
        A(i + length(z),(i-1)*3+3) = 2*x(i);
        A(i + length(z),(i-1)*3-1) = -1;
        A(i + length(z),(i-1)*3) = -2*x(i);
    end
    coeff = A\b
    for i = 1:length(xfine)
        j = find(x>xfine(i),1);
        zfine(i) = coeff(j*3-2) + xfine(i) * coeff(j*3-1) + xfine(i).^2 * coeff(j*3);
    end
end

Created file '/home/martin/Modellbildung-und-Simulation/content/05_interp_approx/quadraticSpline.m'.


Hier bietet es sich an, einen unit test zur Verfügung zu stellen, der diese Funktion auf das erwartete Verhalten testet. 
Es lässt sich zum Beispiel gut überprüfen, ob die Funktion wie erwartet die Interpolationsbedingungen erfüllt oder ob die interpolierten Punkte für Punkte auf einer Parabel wieder auf der ursprünglichen Parabel liegen.

- Testen Sie die Funktionalität mit der bereitgestellten *test suite* `test_quadraticSpline.m`, siehe unten. Es reicht zunächst, dass die ersten n der unit tests erfolgreich sind. **Challenge:** Schaffen Sie es, alle n+i unit tests zu erfüllen?

In [5]:
% test the fac function
moxunit_runtests test_quadraticSpline.m

suite: 7 tests
coeff =

   0.000000
   0.000000
   0.037300
   0.002988
   0.000747
   0.036553
   0.010127
   0.005894
   0.033979
   0.018798
   0.017104
   0.030242
   0.027717
   0.034209
   0.025966

Fcoeff =

   0.000000
   0.000000
   0.037300
   0.002988
   0.000747
   0.036553
   0.010127
   0.005894
   0.033979
   0.018798
   0.017104
   0.030242
   0.027717
   0.034209
   0.025966

Fcoeff =

   0.000000
   0.000000
   0.037300
   0.002988
   0.000747
   0.036553
   0.010127
   0.005894
   0.033979
   0.018798
   0.017104
   0.030242
   0.027717
   0.034209
   0.025966

Fcoeff =

   0.000000
   0.000000
   0.037300
   0.002988
   0.000747
   0.036553
   0.010127
   0.005894
   0.033979
   0.018798
   0.017104
   0.030242
   0.027717
   0.034209
   0.025966

F..coeff =

   0.000000
   0.000000
   0.037300
   0.002988
   0.000747
   0.036553
   0.010127
   0.005894
   0.033979
   0.018798
   0.017104
   0.030242
   0.027717
   0.034209
   0.025966

F
---------------------------

 - Erstellen Sie ein Skript, in welchem Ihre Funktion auf die obigen Messwerte angewandt wird. Stellen Sie den interpolierten Spline bzw. die Biegelinie in einem Plot dar.

In [15]:
%skript to apply function to the given values

%uncomment to equalize axis:
%axis equal
zfine = quadraticSpline([0, 1], [0, 1], 0.5, 0)

coeff =

   0.00000
   0.00000
   0.33333
   0.50000
   0.33333
   0.16667

zfine =  0.70833
